In [1]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import numpy.fft as nf
from scipy import stats
from sklearn.linear_model import LinearRegression

In [2]:
file_array=np.array(['injury_1','injury_2','right_foot_1','right_foot_2','right_foot_3'])


def filter_data(df):
    b, a = signal.butter(3, 0.15, btype='lowpass', analog=False)
    low_passed = signal.filtfilt(b, a, df['aT (m/s^2)'])
    #peaks,_ = signal.find_peaks(df['aT (m/s^2)'])
    #plt.plot(df['time'],low_passed)
    #plt.plot(df['time'],df['aT (m/s^2)'])
    return low_passed


def calculate_vel(df):
    temp=df.shift(periods=-1)
    diff_df=abs(df-temp)
    temp_df=diff_df.shift(periods=1)
    temp_df.iloc[0]=df.iloc[0]
    df['diff_time']=temp_df['time']
    df['velocity']=df['diff_time']*df['aT (m/s^2)']
    return df
    

def calculate_delta_position(df):
    df['position']=df['diff_time']*df['velocity']
    return df

def calculate_total_position(df):
    df2=df
    df2=df2.cumsum(axis=0)
    df['total_position']=df2['position']
    return df


def transform_df(df):
    #using the above function to form the final df we want
    df['aT (m/s^2)']=filter_data(df)
    df=calculate_vel(df)
    df=calculate_delta_position(df)
    df=calculate_total_position(df)
    #we want every 15000 rows for every df so that data will be consistent
    df=df.head(15000)
    return df

    

def get_model(df):
    X=np.stack([df['time']],axis=1)
    X_with=np.concatenate([np.ones(X.shape),X],axis=1)
    model=LinearRegression(fit_intercept=False)
    model.fit(X_with,df['total_position'])
    return model


def split_score_data():
    


In [3]:
#right foot model: 3mins
right=pd.read_csv('right_foot')
right=transform_df(right)
model_right=get_model(right)

,time,ax (m/s^2),ay (m/s^2),az (m/s^2),aT (m/s^2),diff_time,velocity,position,total_position
0,0.003322,-0.2443,-0.1012,0.0000,0.264170,0.003322,0.000878,0.000003,0.000003
1,0.013826,-0.0932,-0.1209,0.0358,0.215172,0.010504,0.002260,0.000024,0.000027
2,0.017828,-0.0424,-0.0818,0.0137,0.171626,0.004002,0.000687,0.000003,0.000029
3,0.027931,0.0769,-0.0426,0.0107,0.137931,0.010103,0.001394,0.000014,0.000043
4,0.038197,0.0784,-0.0624,0.0369,0.116403,0.010266,0.001195,0.000012,0.000056
...,...,...,...,...,...,...,...,...,...
14995,149.956242,-4.1074,1.4907,5.3766,8.197937,0.000762,0.006247,0.000005,16.900922
14996,149.980751,-2.2493,1.2424,0.6013,7.654455,0.024509,0.187603,0.004598,16.905520
14997,149.982363,-3.3206,2.0039,-4.0980,7.335257,0.001612,0.011824,0.000019,16.905539
14998,149.983019,-4.6068,0.2059,-6.2434,7.134204,0.000656,0.004680,0.000003,16.905542
